Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

/home/orfeu/Documents/documents/info_perso/minGPT/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [3]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 124.44M


In [4]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [3]:
def generate_test(use_mingpt, model_type, prompt, device, model, steps=20, do_sample=True):
    
    empty_prompt = False
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            empty_prompt = True
            pass
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            empty_prompt = True
            pass
        else:
            encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
            x = encoded_input['input_ids']
    
    
    
    if not empty_prompt:
        # forward the model `steps` times to get samples, in a batch
        y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

        out = tokenizer.decode(y[0].cpu())
    else:
        out = "no prompt given, plese retry following the instructions above"
    return out

In [2]:
input_prompt = "HR recommendation systems must continuously adapt to evolving candidate profiles and job postings.\
    Static, batch-trained models struggle to stay up-to-date and often fail to maintain relevance over time without costly retraining.\
    This PhD project explores how continual learning (CL) and online learning to rank (OLTR) can enable adaptive, \
    scalable HR systems built on structured JSON data—including parsed resumes and job descriptions. \
    Key research questions include: Incremental adaptation, Real-time ranking, Domain-specific embeddings, \
    Conducted in partnership with HrFlow.ai, this research is anchored in real-world, structured HR datasets and production systems.\
    Expected contributions include new algorithms for lifelong ranking and dynamic representation learning tailored to HR applications.\n\n\
    list of requirements: automnomy, creativity, strong coding skills in python, machine learning, deep learning, NLP, information retrieval, "

In [14]:
input_prompt_test_1 = ""
input_prompt_test_2 = "Hey, I am"

In [1]:
generate_test(use_mingpt=use_mingpt, model_type=model_type, prompt=input_prompt_test_1, device=device, model=model, steps=20, do_sample=True)

NameError: name 'generate_test' is not defined

In [16]:
generate_test(use_mingpt=use_mingpt, model_type=model_type, prompt=input_prompt_test_2, device=device, model=model, steps=20, do_sample=True)

"Hey, I am here to take care of you. It doesn't matter if you're a bad guy, a traitor"

In [11]:
generate(prompt=input_prompt, num_samples=1, steps=10)

--------------------------------------------------------------------------------
HR recommendation systems must continuously adapt to evolving candidate profiles and job postings.    Static, batch-trained models struggle to stay up-to-date and often fail to maintain relevance over time without costly retraining.    This PhD project explores how continual learning (CL) and online learning to rank (OLTR) can enable adaptive,     scalable HR systems built on structured JSON data—including parsed resumes and job descriptions.     Key research questions include: Incremental adaptation, Real-time ranking, Domain-specific embeddings,     Conducted in partnership with HrFlow.ai, this research is anchored in real-world, structured HR datasets and production systems.    Expected contributions include new algorithms for lifelong ranking and dynamic representation learning tailored to HR applications.

    list of requirements: automnomy, creativity, strong coding skills in python, machine learnin